In [1]:
import os

In [2]:
print(os.getcwd())
os.chdir("../")
print(os.getcwd())

c:\Users\A2Z\Desktop\MLOPs\cicd-project\research
c:\Users\A2Z\Desktop\MLOPs\cicd-project


In [3]:
from dataclasses import dataclass 
from pathlib import Path 

In [4]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path 
    test_data_path: Path 
    model_name: Path
    alpha: float
    l1_ratio: float
    target_column: str

In [5]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])



    def get_model_trainer_configuration(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        model_trainer_configuration = ModelTrainerConfig(
            root_dir=config.root_dir, 
            train_data_path=config.train_data_path, 
            test_data_path=config.test_data_path, 
            model_name=config.model_name, 
            alpha=params.alpha, 
            l1_ratio=params.l1_ratio, 
            target_column=schema.name
        )

        return model_trainer_configuration


In [7]:
import os
import pandas as pd
from mlProject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [24]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config



    def train(self):
        try: 
            print(self.config.train_data_path, self.config.test_data_path)
            train_ =  pd.read_csv(self.config.train_data_path)
            test_ = pd.read_csv(self.config.test_data_path)


            train_x = train_.drop([self.config.target_column], axis=1)
            train_y = train_[[self.config.target_column]]

            test_x = test_.drop([self.config.target_column], axis=1)
            test_y = test_[[self.config.target_column]]

            lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42) 
            lr.fit(train_x, train_y)

            joblib.dump(lr, filename=self.config.model_name)

        except Exception as e:
            raise e

In [25]:
try: 
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_configuration()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()

except Exception as e: 
    raise e

[2025-02-03 22:25:24,452: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-03 22:25:24,457: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-03 22:25:24,464: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-03 22:25:24,467: INFO: common: created directory at: artifacts]
[2025-02-03 22:25:24,470: INFO: common: created directory at: artifacts/model_trainer]
artifacts/data_transformation/train.csv artifacts/data_transformation/test.csv
